In [1]:
import pandas as pd
import ads
import requests
import PyPDF2
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
import numpy as np
import random
import re
import urllib3
import time
from requests.adapters import TimeoutSauce

https://ui.adsabs.harvard.edu/help/api/api-docs.html#get-/resolver/-bibcode-/-link_type-
    


In [2]:
ADS_DEV_KEY = "FEbkvybqpxpJQdeF5E8btv7KvfFM4Eh5ezHV7Z8S"
ads.config.token = "FEbkvybqpxpJQdeF5E8btv7KvfFM4Eh5ezHV7Z8S"

In [7]:
# read in database bibcodes

db_bibcodes = pd.read_csv(".//data//db_bibcodes.csv")
db_bibcodes

,Bibcodes
0,1997HIP...C......0E
1,2003A&A...401..781F
2,2003AJ....126.1090C
3,2004ApJS..154...10F
4,2004ApJS..154...25R
...,...
2775,2022A&A...663A.145H
2776,2022ApJS..259...62I
2777,2022AJ....164..198M
2778,2021AJ....161....5W


In [4]:
# get bibcodes from the lit tracker

# auth = ('nsuse', '')

# def isNaN(string):
#     return string != string


# r = requests.get('https://confluence.ipac.caltech.edu/rest/api/content/349470824',
#     params={'expand': 'body.storage,version'},
#     auth=auth)

# json_conf = r.json()

# text = r.json()['body']['storage']['value']

# all_dfs = pd.read_html(text)

# lit_tracker_bibcodes = []
# for i in range(len(all_dfs)):
#     if i == 3 or i == 4 or i==8:
#         bibs = list(all_dfs[i].iloc[:,0])
#     elif i == 16:
#         continue
#     else:
#         bibs = list(all_dfs[i].iloc[:,1])

#     lit_tracker_bibcodes.extend([str(j) for j in bibs if not isNaN(j)])

In [5]:
# save lit tracker bibcodes to 'lit_tracker_bibcodes.txt'

# with open('lit_tracker_bibcodes.txt', 'w') as f:
#     for line in lit_tracker_bibcodes:
#         f.write(f"{line}\n")

In [5]:
# read in lit tracker bibcodes

lit_tracker_bibcodes = []
with open('.//data//lit_tracker_bibcodes.txt', 'r') as f:
    for line in f:
        lit_tracker_bibcodes.append(line.strip())
len(lit_tracker_bibcodes)

1465

In [5]:
# request function for getting PDFs

def download_source_pdf(bibcode, source, folder): # source is "pub" or "eprint"
    headers = {
        'Authorization': f'Bearer {ADS_DEV_KEY}',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
    }
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=10)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(f'https://ui.adsabs.harvard.edu/link_gateway/{bibcode}/{source}_pdf', headers=headers)
    with open(f'.//data//{folder}/{bibcode}.pdf', 'wb') as f:
        f.write(response.content)

In [3]:
# request function for getting other things

def get_url(url):
    headers = {
    'Authorization': f'Bearer {ADS_DEV_KEY}',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
    }
    session = requests.Session()
    
    retry = Retry(connect=3, backoff_factor=10)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(url, headers=headers, timeout=5)
    response.raise_for_status()
    
    return response.json()["response"]["docs"]

In [ ]:
get_url(f'https://api.adsabs.harvard.edu/v1/search/query?q=bibcode:{"2022AJ....164..138S"}&fl=keyword')

In [ ]:
# downloads PDFs from database bibcodes

# directory = ".//data//EA papers//"
# files = os.listdir(directory)
# downloaded_bibs = [f[:-4] for f in files]

# for bibcode in db_bibcodes["Bibcodes"]:
#     if bibcode in downloaded_bibs:
#         print(bibcode, "existing")
#         file_name = bibcode + ".pdf"
#         file_size = os.path.getsize(directory+file_name)
        
#         if file_size < 80000:
#             try:
#                 download_source_pdf(bibcode, "eprint")
#             except:
#                 continue
        
#     else:
#         print(bibcode, "new")
#         download_pub_pdf(bibcode, "pub")
        
#         file_name = bibcode + ".pdf"
#         file_size = os.path.getsize(directory+file_name)
#         if file_size < 80000:
#             try:
#                 download_source_pdf(bibcode, "eprint")
#             except:
#                 continue




In [ ]:
# download PDFs from lit tracker bibcodes

# directory = ".//data//EA papers//"
# files = os.listdir(directory)
# downloaded_bibs = [f[:-4] for f in files]

# for bibcode in lit_tracker_bibcodes:
#     if bibcode in downloaded_bibs:
#         file_name = bibcode + ".pdf"
#         file_size = os.path.getsize(directory+file_name)
        
#         if file_size < 80000:
#             try:
#                 download_EA_source_pdf(bibcode, "eprint")
#             except:
#                 continue
        
#     else:
#         download_EA_source_pdf(bibcode, "pub")
        
#         file_name = bibcode + ".pdf"
#         file_size = os.path.getsize(directory+file_name)
#         if file_size < 80000:
#             try:
#                 download_EA_source_pdf(bibcode, "eprint")
#             except:
#                 continue

In [ ]:
# get negative bibcodes; check if "exoplanet" is not in keywords first
# negative bibcodes are saved to 'nonexoplanet_negatives.txt'

negative_bibcodes = []

# need to iterate through small year chunks manually
year = 2016
while year <= 2022:
    next_year = year + 2
    print(year, next_year)
    
    result = get_url(f'https://api.adsabs.harvard.edu/v1/search/query?&q=database:astronomy&fq=property:refereed&fq=year:[{year} TO {next_year}]&rows=2000&fl=bibcode')
    bibcodes = [i["bibcode"] for i in result if i["bibcode"] not in db_bibcodes and i["bibcode"] not in negative_bibcodes and i["bibcode"] not in lit_tracker_bibcodes] # also not in nonexoplanet_negatives2.txt

    nonexoplanet_negative_bibcodes = []
    for bib in bibcodes:
        response = get_url(f'https://api.adsabs.harvard.edu/v1/search/query?q=bibcode:{bib}&fl=keyword')
        print(response)
        
        if len(response) > 0:
            exoplanet = False

            if "keyword" in response[0].keys():
                keyword = response[0]["keyword"]
                keyword_search = [re.search("planet", i.lower()) for i in keyword]
                if any(keyword_search):
                    exoplanet = True
                    continue

                if not exoplanet:
                    print(bib)
                    nonexoplanet_negative_bibcodes.append(bib)
                    with open('.//data//nonexoplanet_negatives.txt', 'a') as f:
                        f.write(f"{bib}\n")
            else:
                continue
        
        else:
            continue
    
    
    negative_bibcodes.extend(nonexoplanet_negative_bibcodes)
    year += 2

In [4]:
# read in negative bibcodes

negative_bibcodes = []
with open('.//data//nonexoplanet_negatives.txt', 'r') as f:
    for line in f:
        negative_bibcodes.append(line.strip())
        
negative_bibcodes = np.unique(negative_bibcodes) # there are some duplicates
print(len(negative_bibcodes))

checked_nonexoplanet_negative_bibcodes = []
with open('.//data//checked_nonexoplanet_negatives.txt', 'r') as f:
    for line in f:
        checked_nonexoplanet_negative_bibcodes.append(line.strip())
        
print(len(checked_nonexoplanet_negative_bibcodes))

checked_exoplanet_negative_bibcodes = []
with open('.//data//checked_exoplanet_negatives.txt', 'r') as f:
    for line in f:
        checked_exoplanet_negative_bibcodes.append(line.strip())
        
print(len(checked_exoplanet_negative_bibcodes))

9794
9202


FileNotFoundError: [Errno 2] No such file or directory: './/data//checked_exoplanet_negatives.txt'

In [ ]:
# double checking keywords

random.shuffle(negative_bibcodes)

for bib in negative_bibcodes:
    
    if bib in checked_nonexoplanet_negative_bibcodes or bib in checked_exoplanet_negative_bibcodes:
        continue
        
    response = get_url(f'https://api.adsabs.harvard.edu/v1/search/query?q=bibcode:{bib}&fl=keyword')
    print(bib, response)
    
    if len(response) > 0:
        exoplanet = False

        if "keyword" in response[0].keys():
            keyword = response[0]["keyword"]
            keyword_search = [re.search("planet", i.lower()) for i in keyword]
            if any(keyword_search):
                exoplanet = True
                with open('checked_exoplanet_negatives.txt', 'a') as f:
                    f.write(f"{bib}\n")
                continue

            if not exoplanet:
                with open('checked_nonexoplanet_negatives.txt', 'a') as f:
                    f.write(f"{bib}\n")
        else:
            continue

In [ ]:
# download PDFs of negative bibcodes

random.shuffle(negative_bibcodes)

directory = ".//data//non-EA papers//"
files = os.listdir(directory)
downloaded_bibs = [f[:-4] for f in files]

for bibcode in negative_bibcodes:
    if bibcode in downloaded_bibs:
        file_name = bibcode + ".pdf"
        file_size = os.path.getsize(directory+file_name)
        
        if file_size < 80000:
            try:
                download_source_pdf(bibcode, "eprint", "non-EA papers")
            except:
                continue
        
    else:
        download_source_pdf(bibcode, "pub", "non-EA papers")
        
        file_name = bibcode + ".pdf"
        file_size = os.path.getsize(directory+file_name)
        if file_size < 80000:
            try:
                download_source_pdf(bibcode, "eprint", "non-EA papers")
            except:
                continue

In [9]:
# check that there are no lit tracker or db papers in with the negatives

directory = ".//data//non-EA papers//"
files = os.listdir(directory)
downloaded_bibs = [f[:-4] for f in files]
print(len(downloaded_bibs))

for bib in downloaded_bibs:
    if "desktop" in bib:
        continue
    if (bib in lit_tracker_bibcodes) or (bib in db_bibcodes) or (bib not in negative_bibcodes):
        print(bib)
        os.remove(directory+bib+".pdf") 
        
files = os.listdir(directory)
downloaded_bibs = [f[:-4] for f in files]
print(len(downloaded_bibs))

7980
7980
